In [1]:
using Pkg, Logging
Pkg.activate(@__DIR__)

"/home/twan/code/julia-projects/MINLPTrajOpt/notebooks/exploration/Project.toml"

In [2]:
push!(LOAD_PATH, joinpath(@__DIR__, "..", ".."))
push!(LOAD_PATH, joinpath(@__DIR__, "..", "..", "extras"))
Pkg.instantiate()
with_logger(NullLogger()) do
    pkg"precompile"
end

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Precompiling project...


In [3]:
using JuMP
using LinearAlgebra
using Random
using MINLPTrajOpt
using Rotations
using RigidBodyDynamics

In [4]:
urdf = joinpath(dirname(pathof(RigidBodyDynamics)), "..", "test", "urdf", "Acrobot.urdf")
# mechanism = parse_urdf(urdf)
mechanism = parse_urdf(urdf, revolute_joint_type=SinCosRevolute{Float64})

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

In [5]:
using TrigonometricPolynomials
using TrigonometricPolynomials: Var

In [6]:
function symbolic_dynamics(mechanism::Mechanism{X}) where X
    nq = num_positions(mechanism)
    nv = num_velocities(mechanism)
    sincosmap = SinCosDict()
    T = TrigPoly{X}
    state = MechanismState{T}(mechanism)
    q = configuration(state)
    v = velocity(state)
    v̇ = similar(v)
    τ = similar(v)
    for (i, joint) in enumerate(joints(mechanism))
        if joint_type(joint) isa SinCosRevolute
            θjoint = T(Var("θ$i"), sincosmap)
            set_configuration!(state, joint, θjoint)
        else
            qjoint = [T(Var("q$i,$j"), sincosmap) for j = 1 : num_positions(joint)]
            set_configuration!(state, joint, qjoint)
        end
        copyto!(v[joint], [T(Var("v$i,$j"), sincosmap) for j = 1 : num_velocities(joint)])
        copyto!(v̇[joint], [T(Var("v̇$i,$j"), sincosmap) for j = 1 : num_velocities(joint)])
        copyto!(τ[joint], [T(Var("τ$i,$j"), sincosmap) for j = 1 : num_velocities(joint)])
    end
    setdirty!(state)
    M = mass_matrix(state)
    c = dynamics_bias(state)
    # TODO: simplify given quaternion unit norm
    q, v, v̇, τ, M * v̇ + c - τ
end

symbolic_dynamics (generic function with 1 method)

In [7]:
function dynamics_constraints(sym_dynamics::AbstractVector{<:TrigPoly}, varmap::Dict{Var})
    nv = length(sym_dynamics)
    exprs = Vector{Expr}(undef, nv)
    for i in 1 : nv
        sum = Expr(:call, :+)
        for t in terms(sym_dynamics[i].poly)
            c = coefficient(t)
            m = monomial(t)
            prod = Expr(:call, :*, coefficient(t))
            for (x, power) in powers(m)
                if power == 1
                    push!(prod.args, :($(varmap[x])))
                elseif power != 0
                    push!(prod.args, :($(varmap[x])^$(power)))
                end
            end
            push!(sum.args, prod)
        end
        exprs[i] = :($sum == 0)
    end
    exprs
end

dynamics_constraints (generic function with 1 method)

In [8]:
# q, v, v̇, τ, exprs = symbolic_dynamics(mechanism)

In [9]:
# using SymEngine
# Base.sincos(x::Basic) = (sin(x), cos(x))
# q = [symbols("q$i") for i = 1 : num_positions(mechanism)]
# v = [symbols("v$i") for i = 1 : num_velocities(mechanism)]
# state = MechanismState(mechanism, q, v)
# setdirty!(state)
# frame = default_frame(last(bodies(mechanism)))
# p = transform(Point3D(frame, 0, 0, 0), transform_to_root(state, frame));

In [10]:
using RigidBodyDynamics.CustomCollections: SegmentedVector

In [11]:
const JointSegmentedVector{T} = SegmentedVector{JointID, T, Base.OneTo{JointID}, Vector{T}}

SegmentedVector{JointID,T,Base.OneTo{JointID},Array{T,1}} where T

In [12]:
abstract type ObjectiveType end

struct MinEffort <: ObjectiveType end
struct MinTime <: ObjectiveType end

In [23]:
struct TrajOptProblem
    model::JuMP.Model
    Δts::Vector{JuMP.Variable}
    Δqs::Vector{JointSegmentedVector{JuMP.Variable}}
    qs::Vector{JointSegmentedVector{JuMP.Variable}}
    vs::Vector{JointSegmentedVector{JuMP.Variable}}
    v̇s::Vector{JointSegmentedVector{JuMP.Variable}}
    τs::Vector{JointSegmentedVector{JuMP.Variable}}

    function TrajOptProblem(mechanism::Mechanism, x0::MechanismState, solver;
            N::Integer, # number of integration steps
            Δtmin::Number, # min time step
            Δtmax::Number, # max time step
            objectivetype::ObjectiveType,
            T::Union{Number, Nothing} = objectivetype isa MinTime ? nothing : N * Δtmax, # final time
            Δθmax::Number = 0.5, # max angle change during one integration time step (to control badness of small-angle approximation)
        )
        model = Model(solver=solver)
        nq = num_positions(mechanism)
        nv = num_velocities(mechanism)
    
        # Time step settings
        @assert Δtmax >= Δtmin
        fixedstep = Δtmin == Δtmax
        if fixedstep
            T == N * Δtmax || throw(ArgumentError())
        end
    
        # Initial state
        qprev = configuration(x0)
        vprev = velocity(x0)
        
        # Variable vectors
        Δts = Vector{JuMP.Variable}(undef, N)
        Δqs = [similar(qprev, JuMP.Variable) for i = 1 : N]
        qs = [similar(qprev, JuMP.Variable) for i = 1 : N]
        vs = [similar(vprev, JuMP.Variable) for i = 1 : N]
        v̇s = [similar(vprev, JuMP.Variable) for i = 1 : N]
        τs = [similar(vprev, JuMP.Variable) for i = 1 : N]
    
        # Symbolic dynamics
        qsym, vsym, v̇sym, τsym, dynsym = symbolic_dynamics(mechanism)
   
        for i = 1 : N
            # Create variables
            Δts[i] = @variable model basename="Δt_{$i}"
            Δq = Δqs[i] .= [@variable model basename="Δq_{$j,$i}" for j = 1 : nq]
            q = qs[i] .= [@variable model basename="q_{$j,$i}" for j = 1 : nq]
            v = vs[i] .= [@variable model basename="v_{$j,$i}" for j = 1 : nv]
            v̇ = v̇s[i] .= [@variable model basename="v̇_{$j,$i}" for j = 1 : nv]
            τ = τs[i] .= [@variable model basename="τ_{$j,$i}" for j = 1 : nv]
        
            # Time steps
            if fixedstep
                JuMP.fix(Δts[i], Δtmin)
                Δt = Δtmin
            else
                JuMP.setlowerbound(Δts[i], Δtmin)
                JuMP.setupperbound(Δts[i], Δtmax)
                Δt = Δts[i]
            end
        
            for joint in tree_joints(mechanism)
                if joint_type(joint) isa SinCosRevolute
                    # Kinematics delta
                    sΔθ, cΔθ = Δq[joint]
                    setlowerbound.((sΔθ, cΔθ), -1.0)
                    setupperbound.((sΔθ, cΔθ),  1.0)
                    if Δθmax < π / 2
                        @constraint model -sin(Δθmax) <= sΔθ <= sin(Δθmax)
                        if Δθmax < π
                            @constraint model cos(Δθmax) <= cΔθ
                        end
                    end

                    # Absolute kinematics (from rotation matrix product)
                    sθprev, cθprev = qprev[joint]
                    sθ, cθ = q[joint]
                    @NLconstraint model sθ^2 + cθ^2 == 1
                    @NLconstraint model sθ == sθprev * cΔθ + cθprev * sΔθ
                    @NLconstraint model cθ == cθprev * cΔθ - sθprev * sΔθ
                
                    # Velocity
                    Δθ = sΔθ # first-order approximation
                    θd = v[joint][1]
                    @constraint model Δt * θd == Δθ
                else
                    error()
                end
            end
        
            # Acceleration
            @constraint model Δt .* v̇ .== v .- vprev
            
            # Dynamics
            varmap = Dict(Iterators.flatten((
                    zip(variable.(qsym), q),
                    zip(variable.(vsym), v),
                    zip(variable.(v̇sym), v̇),
                    zip(variable.(τsym), τ)
            )))
            JuMP.addNLconstraint.(Ref(model), dynamics_constraints(dynsym, varmap))

            qprev = q
            vprev = v
        end
    
        # TODO: objective, final state constraint
        @objective model Min 1
    
        new(model, Δts, Δqs, qs, vs, v̇s, τs)
    end
end

In [24]:
using BARON

N = 40
objectivetype = MinEffort()
fixedstep = true
T = 4.0

if fixedstep
    Δt = T / N
    Δtmin = Δt
    Δtmax = Δt
else
    Δtmin = 0.0
    Δtmax = 0.5
end

x0 = MechanismState(mechanism)
rand!(x0)
solver = BaronSolver(threads=Sys.CPU_THREADS)
problem = TrajOptProblem(mechanism, x0, solver;
    Δtmin=Δtmin, Δtmax=Δtmax, T=T, N=N, objectivetype=objectivetype);

In [25]:
solve(problem.model)

 BARON version 18.8.23. Built: LNX-64 Thu Aug 23 14:46:44 EDT 2018

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: MIT at Twan Koolen, tkoolen@mit.edu.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Khajavirad, A. and N. V. Sahinidis,
 A hybrid LP/NLP paradigm for global optimization relaxations,
 Mathematical Programming Computation, 10, 383-421, 2018.
 This BARON run may utilize the following subsolver(s)
 For LP/MIP: ILOG CPLEX                                      
 For NLP: IPOPT, FILTERSD, FILTERSQP
 Doing local search
 Unable to find/load CPLEX library libcplex.so: cannot open shared object file: No such file or directory
 Using CPLEX library libcplex1280.so.
 Preprocessing found feasible solution with value  1.00000000000     
 Problem solved during preprocessing
 Lower bound is  1.00000000000     

 Cleaning up

                         *** Normal

:Optimal